In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
import tensorflow_hub as hub

In [ ]:
# 한번에 하나의 이미지만 예측할 수 있다.

def Predict_CXR(image_path, feature_model_path, model_path):
    # 모델 불러오기
    # 모델 불러올 때 Warning이 표시되는데 이는 더 training하지 못한다는 의미이고,
    # 이 함수에서는 더 이상 training을 하지 않음.
    model = load_model(model_path)
    feature_model = load_model(feature_model_path, custom_objects={'KerasLayer': hub.KerasLayer})

    # 이미지를 불러옴
    img = cv2.imread(image_path)
    # 이미지 크기를 모델에 맞게 조정
    img = cv2.resize(img, dsize=(224,224))
    # 이미지 픽셀값 스케일링
    img = img / 255.0
    # 이미지 차원 확장 (224,224,3) -> (1,224,224,3)  : feature_model에 넣기위함
    img = np.expand_dims(img, axis=0)
    # feature_model에서 feature 추출
    feature_vector = feature_model.predict(img)
    # 앞서 생성한 model 분류기를 통해 예측 수행
    prediction = model.predict(feature_vector)[0]

    unique_sorted_Y = ['COVID19','NORMAL','PNEUMONIA']

    # 가장 예측값이 높은 인덱스를 반환
    index = prediction.argmax()

    # labels에 저장 
    label = unique_sorted_Y[index]
    print(label)
    return label
Predict_CXR('/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/COVID-19/COVID-19 (10).png',
            '/content/drive/My Drive/인공지능 학습관련/covid19_detection_model/feature_model.h5',
            '/content/drive/My Drive/인공지능 학습관련/covid19_detection_model/Inception_Resnet_V2_224_best.h5')

COVID19


In [ ]:
paths = ['/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/COVID-19/COVID-19 (1).png',
         '/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/COVID-19/COVID-19 (100).png',
         '/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/NORMAL/NORMAL (1).png',
         '/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/NORMAL/NORMAL (1004).png',
         '/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/Viral Pneumonia/Viral Pneumonia (10).png',
         '/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/Viral Pneumonia/Viral Pneumonia (1005).png']

# 여러 사진들을 한 번에 예측을 수행한다.
# feature_model과 model을 반복마다 부르지 않아도 된다.

def Predict_CXR_iter(image_path_list, feature_model_path, model_path):
    # 모델 불러오기
    # 모델 불러올 때 Warning이 표시되는데 이는 더 training하지 못한다는 의미이고,
    # 이 함수에서는 더 이상 training을 하지 않음.
    model = load_model(model_path)
    feature_model = load_model(feature_model_path, custom_objects={'KerasLayer': hub.KerasLayer})

    labels =[]
    for image_path in image_path_list:
        # 이미지를 불러옴
        img = cv2.imread(image_path)
        # 이미지 크기를 모델에 맞게 조정
        img = cv2.resize(img, dsize=(224,224))
        # 이미지 픽셀값 스케일링
        img = img / 255.0
        # 이미지 차원 확장 (224,224,3) -> (1,224,224,3)  : feature_model에 넣기위함
        img = np.expand_dims(img, axis=0)
        # feature_model에서 feature 추출
        feature_vector = feature_model.predict(img)
        # 앞서 생성한 model 분류기를 통해 예측 수행
        prediction = model.predict(feature_vector)[0]  

        unique_sorted_Y = ['COVID19','NORMAL','PNEUMONIA']
        # 가장 예측값이 높은 인덱스를 반환
        index = prediction.argmax()

        # labels에 저장 
        label = unique_sorted_Y[index]
        print(label)
        labels.append(label)
    return labels
Predict_CXR_iter(paths,
        '/content/drive/My Drive/인공지능 학습관련/covid19_detection_model/feature_model.h5',
        '/content/drive/My Drive/인공지능 학습관련/covid19_detection_model/Inception_Resnet_V2_224_best.h5')

COVID19
COVID19
NORMAL
NORMAL
PNEUMONIA
PNEUMONIA


In [ ]:
model = load_model('/content/drive/My Drive/인공지능 학습관련/covid19_detection_model/Inception_Resnet_V2_224_best.h5')
feature_model = load_model('/content/drive/My Drive/인공지능 학습관련/covid19_detection_model/feature_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
# 한번에 하나의 이미지만 예측할 수 있다.
# 모델을 함수 밖에서 불러온다면? --> 함수 실행시마다 불러오지 않고 한 번 불러오면 계속 사용가능.
# views.py 에서 이미 모델과 피쳐모델이 불려져 있다면 바로 이 함수를 실행시킬 수 있다.
def Predict_CXR(image_path):
    
    # 이미지를 불러옴
    img = cv2.imread(image_path)
    # 이미지 크기를 모델에 맞게 조정
    img = cv2.resize(img, dsize=(224,224))
    # 이미지 픽셀값 스케일링
    img = img / 255.0
    # 이미지 차원 확장 (224,224,3) -> (1,224,224,3)  : feature_model에 넣기위함
    img = np.expand_dims(img, axis=0)
    # feature_model에서 feature 추출
    feature_vector = feature_model.predict(img)
    # 앞서 생성한 model 분류기를 통해 예측 수행
    prediction = model.predict(feature_vector)[0]
    print(prediction)
    
    unique_sorted_Y = ['COVID19','NORMAL','PNEUMONIA']
    #확률의 예측값을 5개 선출 
    # 가장 예측값이 높은 인덱스를 반환
    index = prediction.argmax()

    # labels에 저장 
    label = unique_sorted_Y[index]
    print(label)
    return label

Predict_CXR('/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/COVID-19/COVID-19 (10).png')

[9.9997342e-01 1.2245206e-05 1.4288857e-05]
COVID19
